# 1. Chuẩn bị dữ liệu (data preparation)

**Thư mục ảnh của bạn đặt trong thư mục datasets/train và datasets/test:**
- Đặt tên 'dog_n' với những ảnh label là 'dog' (ví dụ: dog_1.jpeg, dog_2.jpeg...)
- Đặt tên 'n' với những ảnh lable là 'non-dog' (ví dụ: 1.jpeg, 2.jpeg...)
- Quy cách đặt tên áp dụng cho cả 2 thư mục train và test

**Tạo thư mục output:**
- Thư mục này sẽ chưa file *.h5 của bạn.

In [24]:
from random import shuffle
import glob

hdf5_train_path = 'output/train_dogvnondog.h5'  # file path for the created .hdf5 file
hdf5_test_path = 'output/test_dogvnondog.h5'

cat_dog_train_path = './datasets/train/*.jpg' # the original data path
cat_dog_test_path = 'datasets/test/*.jpg' 

# get all the image paths 
train_addrs = glob.glob(cat_dog_train_path)
print(len(train_addrs))
test_addrs = glob.glob(cat_dog_test_path)
print(len(test_addrs))

# label the data as 1=dog, 0=nondog
train_labels = [1 if 'dog' in addr else 0 for addr in train_addrs] 
test_labels = [1 if 'dog' in addr else 0 for addr in test_addrs] 

237
66


# 2. Tạo file h5 Object

In [25]:
import numpy as np
import h5py

train_shape = (len(train_addrs), 64, 64, 3)
test_shape = (len(test_addrs), 64, 64, 3)

# open a hdf5 file and create earrays 
f_train = h5py.File(hdf5_train_path, mode='w')
f_test = h5py.File(hdf5_test_path, mode='w')

# create the "list_classes" for training dataset
f_train.create_dataset("list_classes", (2,), 'S7')
class_0 = 'non-dog'
class_1 = 'dog'
f_train["list_classes"][0]= class_0.encode("ascii", "ignore")
f_train["list_classes"][1]= class_1.encode("ascii", "ignore")

# create the "train_set_x" for training dataset
f_train.create_dataset("train_set_x", train_shape, np.uint8)

# create the "train_set_y" for training dataset
f_train.create_dataset("train_set_y", (len(train_addrs),), np.uint64)
f_train["train_set_y"][...] = train_labels

# create the "list_classes" testing dataset
f_test.create_dataset("list_classes", (2,), 'S7')
class_0 = 'non-dog'
class_1 = 'dog'
f_test["list_classes"][0]= class_0.encode("ascii", "ignore")
f_test["list_classes"][1]= class_1.encode("ascii", "ignore")

# create the "test_set_x" dataset
f_test.create_dataset("test_set_x", test_shape, np.uint8)

# create the "test_set_y" dataset
f_test.create_dataset("test_set_y", (len(test_addrs),), np.uint64)
f_test["test_set_y"][...] = test_labels

# 3. Ghi các hình ảnh vào file h5

In [26]:
import cv2

# loop over train paths
for i in range(len(train_addrs)):
  
    if i % 1000 == 0 and i > 1:
        print ('Train data: {}/{}'.format(i, len(train_addrs)) )

    addr = train_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    f_train["train_set_x"][i, ...] = img[None] 

f_train.close()

# loop over test paths
for i in range(len(test_addrs)):
  
    if i % 1000 == 0 and i > 1:
        print ('Test data: {}/{}'.format(i, len(test_addrs)) )

    addr = test_addrs[i]
    img = cv2.imread(addr)
    img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
    f_test["test_set_x"][i, ...] = img[None] 

f_test.close()


# 4.Good luck

!!!!!